In [ ]:
pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 63.4 MB/s eta 0:00:00


In [40]:
import os
import pandas as pd

from gensim.models import Word2Vec, Phrases
from gensim.models.fasttext import FastText
from pyvi import ViTokenizer
from gensim.models.phrases import Phrases, Phraser

In [ ]:
pathdata = '/content/drive/MyDrive/Data_full/merged_data (2).txt'
stopwords_path = '/content/drive/MyDrive/Data_full/stopwords.csv'

model_path = '/content/word2vec.model'
fast_text = '/content/fasttext_gensim.model'

In [ ]:
# Đọc dữ liệu từ file
train_data = pd.read_csv(pathdata, delimiter='\t', header=None, names=['text'])
train_data['text_tokenized'] = train_data['text'].apply(ViTokenizer.tokenize)
train_data

,text,text_tokenized
0,Việt Nam quốc hiệu là Cộng hòa Xã hội chủ ngh...,Việt_Nam quốc_hiệu là Cộng_hòa Xã_hội chủ_nghĩ...
1,Lãnh thổ Việt Nam xuất hiện con người sinh sốn...,Lãnh_thổ Việt_Nam xuất_hiện con_người sinh_sốn...
2,Đến thời kỳ cận đại Việt Nam lần lượt trải qu...,Đến thời_kỳ cận_đại Việt_Nam lần_lượt trải qua...
3,Sau khi thống nhất Việt Nam tiếp tục gặp khó ...,Sau khi thống_nhất Việt_Nam tiếp_tục gặp khó_k...
4,Tên gọi,Tên gọi
...,...,...
12232,Y học Gerhard Domagk,Y_học Gerhard_Domagk
12233,Văn học Frans Eemil Sillanpää,Văn_học Frans Eemil_Sillanpää
12234,Hòa bình không có giải,Hòa_bình không có giải
12235,Xem thêm,Xem thêm


In [47]:
# Đọc danh sách stopwords
stopwords_data = pd.read_csv(stopwords_path, sep="\t", encoding='utf-8')
stopwords_set = set(stopwords_data['stopwords'])
stopwords_data['stopwords'].head()

0      mà_thôi
1       tuy_có
2      đâu_như
3    quay_bước
4      hỏi_lại
Name: stopwords, dtype: object

In [ ]:
# Tiền xử lý dữ liệu
train_data['text_cleaned'] = train_data['text_tokenized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords_set]))
train_data['text_cleaned']

0        Việt_Nam quốc_hiệu Cộng_hòa Xã_hội chủ_nghĩa V...
1        Lãnh_thổ Việt_Nam con_người sinh_sống thời_đại...
2        Đến thời_kỳ cận_đại Việt_Nam lần_lượt trải gia...
3        Sau thống_nhất Việt_Nam sụp_đổ tan_rã đồng_min...
4                                                  Tên gọi
                               ...                        
12232                                 Y_học Gerhard_Domagk
12233                        Văn_học Frans Eemil_Sillanpää
12234                                        Hòa_bình giải
12235                                                  Xem
12236                                            Tham_khảo
Name: text_cleaned, Length: 12237, dtype: object

In [ ]:
# Chuyển dữ liệu huấn luyện thành danh sách các từ
sentences = [text.split() for text in train_data['text_cleaned']]

### Word2Vec

In [ ]:
# Xử lý các cặp từ thông dụng hoặc cụm từ
bigram_transformer = Phraser(Phrases(sentences, min_count=5, threshold=100))
sentences_with_phrases = list(bigram_transformer[sentences])

In [ ]:
# Xây dựng mô hình Word2Vec với Negative Sampling
model_word2vec = Word2Vec(sentences_with_phrases, vector_size=150, window=10, min_count=10, workers=4, sg=1, negative=5)
model_word2vec.save(model_path)  # Lưu mô hình tại đường dẫn đã chỉ định

In [ ]:
# Hiển thị từ có độ tương đồng cao nhất
for word, score in model_word2vec.wv.most_similar("Hồ_Chí_Minh"):
    print(word, score)

Nguyễn_Ái_Quốc 0.9767552018165588
Việt_Nam 0.9741204977035522
Hội 0.9726068377494812
cách_mạng 0.9721933603286743
Cộng_sản 0.9665721654891968
Ông 0.9664304256439209
tham_gia 0.9638090133666992
dân_tộc 0.9627223610877991
thuộc_địa 0.9624274969100952
Đông_Dương 0.9621381759643555


###fastText

In [ ]:
# Xây dựng mô hình FastText
model_fasttext = FastText(sentences_with_phrases, vector_size=150, window=10, min_count=10, workers=4, sg=1, negative=5)
model_fasttext.save(fast_text)  # Lưu mô hình tại đường dẫn đã chỉ định

In [ ]:
# Hiển thị từ có độ tương đồng cao nhất
for word, score in model_fasttext.wv.most_similar("Hồ_Chí_Minh"):
    print(word, score)

lập 0.99101722240448
chủ_tịch 0.9883866906166077
chủ_nghĩa_cộng_sản 0.9880251288414001
cộng_sản 0.9879514575004578
Nguyễn_Ái_Quốc 0.9878663420677185
dân_chủ 0.987087607383728
Xã_hội 0.9852777123451233
xã_hội_chủ_nghĩa 0.9849224090576172
Chủ_nghĩa 0.9845078587532043
Viện 0.9833997488021851


In [ ]:
'mongkut' in train_data['text_cleaned'].str.split().values.tolist()

False